# Exact Solution for Job Shop Problem

Данная модель находит решение JSP с помощью вспомогательной утилиты cp_model из пакета Google Operation Research, которая позволяет организовать перебор из области допустимых значений учитывая ограничивающие условия. Также с ее помощью можно найти такие допустимые значения при которых наперед заданая функция достигает минимального (максимального) значения. Именно этими фунциями библиотеки мы и будем пользоваться.

In [39]:
import collections
from ortools.sat.python import cp_model

### Условия задачи

In [40]:
jobs_data = [  # Подзадача = (номер_машины, время_выполнения).
        [(0, 2), (1, 3), (3, 4), (2, 3)],  # Задача 0
        [(1, 5), (2, 2), (0, 3), (1, 3)],  # Задача 1
        [(1, 4), (0, 2), (2, 1), (3, 1)],  # Задача 2
        [(3, 1), (2, 2), (0, 1), (1, 1)],  # Задача 3
    ]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

# Оценка сверху для области значения всех переменных (сумма всего времени выполнения)
horizon = sum(task[1] for job in jobs_data for task in job)

Обьявляем переменные start и end для каждой подзадачи, и переменную интервала, которая работает, как связь между start и end вида:

**start + duration = end**

In [41]:
model = cp_model.CpModel()
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

task_type = collections.namedtuple('task_type', 'start end interval')
assigned_task_type = collections.namedtuple('assigned_task_type', 'start job index duration')

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
        # Создаем лист, ключем которого будет кортеж (джоб_айди, таск,_айди),а значениям будет именованный кортеж
        # с обьектами переменных (начало, конец, интервал)
        all_tasks[job_id, task_id] = task_type(
            start=start_var, end=end_var, interval=interval_var)
        # Каждой машине ставим в соотвествие интервал таска
        machine_to_intervals[machine].append(interval_var)


*В нашей задаче есть следующие ограничения:*
- Для каждой подзадачи время начала подзадачи + ее длительность меньше или равно времении начала следующей подзадачи 
- Интервалы работы подзадач каждой машины не пересекааются 

In [42]:
# Ограничение на непересечение интервалов
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# Ограничение на конец n-ой и начало n+1-ой подзадачи в одной задаче
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)


Добавим целевую переменную, которую будем затем минимизировать. Это будет максимум концов подзадач.

In [43]:
# Переменная временных затрат
obj_var = model.NewIntVar(0, horizon, 'makespan')

# Добавляем условие obj_var == max(all_tasks.end)
model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
# Минимизируем временные затраты
model.Minimize(obj_var)

# Запускаем перебор
solver = cp_model.CpSolver()
status = solver.Solve(model)

### Вывод результатов

In [44]:
# Если нашли оптимальное решение
if status == cp_model.OPTIMAL:
    # Создаем для каждой машины лист тасков и оптимального решения
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(
                    start=solver.Value(all_tasks[job_id, task_id].start),
                    job=job_id,
                    index=task_id,
                    duration=task[1]))
    # Вывод для задачи
    output = ''
    for machine in all_machines:
        # Сортируем по времени
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Машина ' + str(machine) + ': '
        sol_line = '          '
        
        for assigned_task in assigned_jobs[machine]:
            name = 'подзадача_%i_%i' % (assigned_task.job, assigned_task.index)
            sol_line_tasks += '%-20s' % name
            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            sol_line += '%-20s' % sol_tmp
            
        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    print('Время оптимального решения: %i' % solver.ObjectiveValue())
    print('\n')
    print(output)

Время оптимального решения: 17


Машина 0: подзадача_0_0       подзадача_3_2       подзадача_1_2       подзадача_2_1       
          [0,2]               [3,4]               [7,10]              [12,14]             
Машина 1: подзадача_1_0       подзадача_0_1       подзадача_2_0       подзадача_3_3       подзадача_1_3       
          [0,5]               [5,8]               [8,12]              [12,13]             [13,16]             
Машина 2: подзадача_3_1       подзадача_1_1       подзадача_0_3       подзадача_2_2       
          [1,3]               [5,7]               [12,15]             [15,16]             
Машина 3: подзадача_3_0       подзадача_0_2       подзадача_2_3       
          [0,1]               [8,12]              [16,17]             

